In [3]:
from functools import reduce
from ff import base_repr, ff_for_prime, ff_for_primitive_polynomial, mult_poly

# BCH codes
### Motivation
We start with generalized Hamming codes - $(n = 2^m - 1, k = 2^m - m - 1)$ linear codes that can correct single errors. Steps:
 1. Take the parity-check matrix - its $n$ columns are all the non-zero elements of $V_m = GF(2)^m$ vector space in arbitrary order.
 2. We have $n$ possible locations of a single error, so we need syndromes with $m$ positions to identify each of them. If we want to identiy locations of two errors, then intuitively we'll need 2x as many parity-check rows - the former $m$ rows identify the one error, the latter $m$ rows identiy the other error.
 3. Since the 1st halves of parity-check matrix columns are unique, we can look at the second halves as values of a function $f: V_m \rightarrow V_m$ on the first halves.
 4. Now we just need to choose the function so that for any 2-position error vector we have a unique syndrome - then we can identify and correct the errors. But the syndromes are combinations of parity-check matrix columns! We arrive at a system of equations:
$$
\begin{align}
 u + v = s_1 \\
 f(u) + f(v) = s_2
\end{align}
$$
for all $u, v, s_1, s_2 \in GF(2^m)$ that is supposed to have at most one solution $u, v$. The most simple such function is $f(x) = x^3$.

It's parity-check matrix is
$$
H_2 = \begin{bmatrix}
 \alpha_0 && ... && \alpha_{n - 1} \\
 \alpha_0^3 && ... && \alpha_{n - 1}^3
\end{bmatrix}
$$
where $\alpha_0, ..., \alpha_{n-1}$ is any ordering of the $GF(2^m)$ field's non-zero elements. It has $2m$ rows as elements in $GF(2)$, so the dimension of the rowspace is $\leq 2m$, hence code dimension (length of the information word) is $\geq n - 2m$.

### Generalization
(theorem 9.1)

If we choose $n = 2^m$ - length of the codeword, and $t \leq \lfloor \frac{n - 1}{2} \rfloor$ - max number of errors to be corrected, then the $t \ x \ n$ matrix
$$
H_t = \begin{bmatrix}
 \alpha_0 && ... && \alpha_{n - 1} \\
 \alpha_0^3 && ... && \alpha_{n - 1}^3 \\
 && ... && \\
 \alpha_0^{2t - 1} && ... && \alpha_{n - 1}^{2t - 1}
\end{bmatrix}
$$

is the parity-check matrix of an $(n, k)$ code that can correct all error patterns of weight $\leq t$ and $k \geq n - tm$. These codes are called **BCH codes**.

### Cyclic BCH codes
BCH codes can be made cyclic by using $\alpha_0 = 1, \alpha_1 = \alpha ..., \alpha_{n - 1} = \alpha^{n-1}$, where $\alpha$ is the generator of non-zero elements of the $GF(2^m)$ field. Then we get nice encoders using shift registers for free. Decoding still needs more machinery to be developed since we've only seen burst error correction decoders so far and these codes are supposed to correct ANY error pattern of weight $\leq t$.

### Cyclic BCH code generator polynomial
From the form of $H_t$ it follows, that all the codewords (seen as polynomials $C_0 + ... + C_{n - 1} x^{n - 1}$ over $GF(2)$) satisfy $C(\alpha^i) = 0, \ i = 1, ..., 2t - 1$. The generator polynomial is the lowest degree monic polynomial that also satisfies this condition, so it's the minimal polynomial of the set $\{ \alpha, ..., \alpha^{2t - 1} \}$. Reasoning from the Appendix C shows that it will be a product of binomials with **conjugates** of these elements as roots. 

Phew, that's a mouthful - an example is needed!

### BCH generator polynomial example
Let's start by building a generator polynomial for BCH with given properties.

Following section 9.2 we know that for codeword length $n = 15$, correction of up to $t = 3$ errors the generator polynomial is going to be a product of minimal polynomials for $\alpha$, $\alpha^3$ and $\alpha^5$ (up to power of $5 = 2 t - 1$), where $\alpha$ is the primitive root of GF(16). Hence it's going to be the product of all binomials with roots being conjugate to these three, i.e. $\{\alpha, \alpha^2, \alpha^3, \alpha^4, \alpha^5, \alpha^6, \alpha^8, \alpha^9, \alpha^{10}, \alpha^{12}\}$.

According to the book the result is $g(x) = 1 + x + x^2 + x^4 + x^5 + x^8 + x^{10}$, let's see.

In [4]:
f_2 = ff_for_prime(2)
f_16_prim_pow = ff_for_primitive_polynomial(2, f_2, [1, 1, 0, 0, 1])

poly = [1]
for power in [1, 2, 3, 4, 5, 6, 8, 9, 10, 12]:
    # multiply by x - alpha^power
    poly = mult_poly(poly, [f_16_prim_pow.add_inv(f_16_prim_pow.power_to_num[power]), 1], f_16_prim_pow)

assert poly == [1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1]

Hell yeah, good to confirm again that field operations work fine!

So if we just implement finding conjugates for given powers of primitive element (which is simple), then we can automatically derive BCH generator polynomials for given code parameters $n, \ t$. At the moment, based on such polynomials we can at least implement encoding by generating appropriate generator matrix for cyclic code (or using shift registers).

### BCH generator polynomial automated derivation
Given a finite field $F_{p^m}$ and its element $a$, **conjugates** of $a$ are the unique elements of the sequence $a, a^p, ..., a^{p^{k - 1}}$ and their number $k$ is called the degree of $a$. $a$ belongs to the subfield $F_{p^k}$ but no smaller subfield.

In [5]:
# find conjugates of alpha in F_{p^m} where p is the characteristic of the field
def conjugates(alpha, p, field):
    result = [alpha]
    alpha_p = reduce(lambda x, y: field.mult(x, y), [alpha] * p)  # alpha^p
    alpha_pow = alpha_p

    while alpha_pow != alpha:
        result.append(alpha_pow)
        alpha_pow = field.pow(alpha_pow, p)
    
    return set(result)

# according to the example, the conjugates of alpha are [alpha, alpha^2, alpha^4, alpha^8] and conjugates of alpha^3 are [alpha^3, alpha^6, alpha^12, alpha^9]
expected = {f_16_prim_pow.power_to_num[p] for p in [1, 2, 4, 8]}
assert conjugates(f_16_prim_pow.power_to_num[1], 2, f_16_prim_pow) == expected

expected_3 = {f_16_prim_pow.power_to_num[p] for p in [3, 6, 12, 9]}
assert conjugates(f_16_prim_pow.power_to_num[3], 2, f_16_prim_pow) == expected_3

In [6]:
# p: the field characteristic
# field: GF(p^m)
# alpha: field primitive root
# t: max number of errors our code is able to correct
def bch_generator_polynomial(p, m, field, alpha, t):
    assert t <= (p ** m - 1) / 2

    # generate polynomial roots alpha, ..., alpha^{2t - 1]
    alpha_2 = field.pow(alpha, 2)
    next_root = alpha
    base_roots = []
    
    for _ in range(1, 2 * t + 1, 2):
        base_roots.append(next_root)
        next_root = field.mult(next_root, alpha_2)

    # find conjugates of the roots
    conjugate_sets = [conjugates(root, p, field) for root in base_roots]
    conjugates_final_set = reduce(lambda x, y: x | y, conjugate_sets)

    gen_poly = [1]
    for c in conjugates_final_set:
        gen_poly = mult_poly(gen_poly, [field.add_inv(c), 1], field)  # multiply all (x - conjugate) binomials

    return gen_poly

In [7]:
assert bch_generator_polynomial(p=2, m=4, field=f_16_prim_pow, alpha=f_16_prim_pow.power_to_num[1], t=3) == [1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1]

Hell yeah! So we can automatically generate generator polynomials for BCH codes of selected properties!

# Decoding BCH codes
Well, of all the stuff related to finite fields, to linear and cyclic codes - this is the pinnacle. There is just so much crazy machinery that jumps out of nowhere, like discrete Fourier transforms on vectors over a Galois field - how did they come up with that?? Or *the key equation* that magically can be solved for the *error-locator* and *error-evaluator* polynomials, from which we can compute both error bits and original codeword values.

This is an attempt to make sense of all this theory and see the bigger picture with underlying principles. Hopefully it's gonna make more sense as we go. Luckily, once the tools are understood and developed here, they can be used for Reed-Solomon codes as well.

### DFT
Let's start with DFT of a vector of length $n$ over any finite field with $\alpha$ such that $\alpha^n = 1$ (nth primitive root of unity, i.e. smallest such $n$). 

Note that this definition includes inverse DFT, which doesn't seem to work if the vector length is divisible by field characteristic, e.g. for standard $GF(2^m)$ fields we'll need the length to be odd. But $n$ being the smallest integer s.t. $\alpha^n = 1$ means that $n | q - 1$ (field order) so it should always work.

In [32]:
# polynomial over given field
# alpha: primitive root of the field
def dft(polynomial, alpha, field):
    result = []
    
    for k in range(len(polynomial)):
        # DFT(polynomial)_k = polynomial(alpha^k)
        # could also be optimized by reusing powers of alpha^k but it's not that important
        alpha_k = field.pow(alpha, k)
        poly_value = reduce(lambda x, y: field.add(x, y), [field.mult(c_i, field.pow(alpha_k, i)) for i, c_i in enumerate(polynomial)])
        result.append(poly_value)

    return result


def idft(polynomial, alpha, field):
    # IDFT(polynomial)_k = (1 / n) DFT(polynomial)(alpha^{-k})
    poly_d = dft(polynomial, field.mult_inv(alpha), field)
    n_inv = field.mult_inv(reduce(lambda x, y: field.add(x, y), [1] * len(polynomial)))  # only exists for n not divisble by field characteristic
    return [field.mult(n_inv, pd_i) for pd_i in poly_d]

Let's veryify with the book - example 9.2.

In [33]:
alpha_f_16 = f_16_prim_pow.power_to_num[1]

v = [0, 0, f_16_prim_pow.power_to_num[2], 0, 0, 0, 0, f_16_prim_pow.power_to_num[7], 0, 0, 0, 0, 0, 0, 0]
v_d = dft(v, alpha_f_16, f_16_prim_pow)

polynomial_s = []
for i, v_di in enumerate(v_d):
    r = ''
    if v_di == 0:
        r = v_di
    else:
        power = f_16_prim_pow.num_to_power[v_di]
        if power == 0:
            r = '1'
        else:
            r = f'a^{power}' if power != 1 else 'a'
    polynomial_s.append(r)
assert polynomial_s == ['a^12', 'a^9', 0, 'a^3', '1', 0, 'a^9', 'a^6', 0, '1', 'a^12', 0, 'a^6', 'a^3', 0]

assert idft(v_d, alpha_f_16, f_16_prim_pow) == v

Cool!

### Key equation and all the polynomials
For a given vector $V = (V_0, ..., V_{n - 1})$ over the field $F$ we define a bunch of stuff.

**support set I** - all indexes $i \in \{0, ..., n - 1\}$ where $V_i \neq 0$

**locator polynomial** $\sigma_V(x) = \prod_{i \in I} (1 - \alpha^i x)$

**punctured locator polynomial** $\sigma_V^{(j)}(x) = \prod_{i \in I, \ i \neq j} (1 - \alpha^i x)$

**error evaluator polynomial** $w_V(x) = \sum_{i \in I} V_i \sigma_V^{(i)}(x)$

So beautiful, eh? I mean, they have some properties that make sense, like $\sigma_V(x) = 0 \iff x = \alpha^{-i}, \ i \in I$ and $w_V(x)$ kinda resembles Lagrange interpolation. But I still have no idea where this is going.

And the **key equation** doesn't help much at first sight:
$$
\sigma(x) \hat{V}(x) = w_V(x) (1 - x^n)
$$

It's actually quite simple to prove it! And maybe the **1st corollary** helps a bit, we can express initial vector's coefficients using these polynomials:
$$
V_i = - \alpha^i \frac{w_V(\alpha^{-i})}{\sigma_V '(\alpha^{-i})}, \ i \in I
$$
We use a formal derivative (defined for polynomials over any field, e.g. $(x^4)' := 4 x^3$, where $4 = 1 + 1 + 1 +1$) because it wasn't fun enough already. Or rather because key equation for $\alpha^i, \ i \in I$ zeroes on both sides, but we can take a formal derivative and get a useful result.

**2nd corollary** is that if $\sigma_V(x) = 1 + \sigma_1 x + ... + \sigma_d x^d$ then
$$
\hat{V}_j = - \sum_{i = 1}^d \sigma_i \hat{V}_{j - i}, \  j = 0, ..., n - 1
$$
where the indexes are taken $(mod \ n)$. So this means that if we have the locator polynomial $\sigma_V(x)$ and just $d$ subsequent DFT coefficients, we can recover all the DFT coefficients - interesting...

# TODO:
 - read how to decode these bastards (with error correction of course!) + implement decoder derivation (time & frequency domain, WTF)
 - move this shit elsewhere - this are not RDS fundamentals anymore!